In [188]:
import re
import time
import numpy as np
import cv2
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import cv2
from mrcnn.visualize import display_instances
import matplotlib.pyplot as plt
from mrcnn.config import Config
from mrcnn import utils
from mrcnn import visualize
import shutil



In [189]:
# create a folder and copy images to train, test and validate folder
def create_folder(folder_name):
    try:
        if not os.path.exists(folder_name):
            os.makedirs(folder_name)
    except OSError:
        print ('Error: Creating directory. ' + folder_name)

In [190]:
# function to train, test and validate 
def train_test_validate_split(dataset, train_percent=.6, validate_percent=.2, seed=None):
    np.random.seed(seed)
    perm = np.random.permutation(dataset.shape[0])
    m = dataset.shape[0]
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = dataset[perm[:train_end]]
    validate = dataset[perm[train_end:validate_end]]
    test = dataset[perm[validate_end:]]
    return train, validate, test


print("Loading images ...")
# load the images
crack_images = os.listdir("dataset/crack/accepted")
spall_images = os.listdir("dataset/spall/accepted")
print("Number of crack images: ", len(crack_images))
print("Number of spall images: ", len(spall_images))


# if the images are not in train, test and validate folder, then copy them
if not os.path.exists("dataset/crack/accepted/train"):
    print("Copying images to train, test and validate folder ...")
    # create a folder to store the images
    crack_train = create_folder("dataset/crack/accepted/train")
    crack_validate = create_folder("dataset/crack/accepted/validate")
    crack_test = create_folder("dataset/crack/accepted/test")
    spall_train = create_folder("dataset/spall/accepted/train")
    spall_validate = create_folder("dataset/spall/accepted/validate")
    spall_test = create_folder("dataset/spall/accepted/test")

    # if no images in train, test and validate folder then copy the images
    if len(os.listdir("dataset/crack/accepted/train")) == 0:    
        # copy images to train, test and validate folder
        for i in range(len(crack_images)):
            if i < 0.6 * len(crack_images):
                shutil.copy("dataset/crack/accepted/" + crack_images[i], "dataset/crack/accepted/train")
            elif i < 0.8 * len(crack_images):
                shutil.copy("dataset/crack/accepted/" + crack_images[i], "dataset/crack/accepted/validate")
            else:
                shutil.copy("dataset/crack/accepted/" + crack_images[i], "dataset/crack/accepted/test")

        for i in range(len(spall_images)):
            if i < 0.6 * len(spall_images):
                shutil.copy("dataset/spall/accepted/" + spall_images[i], "dataset/spall/accepted/train")
            elif i < 0.8 * len(spall_images):
                shutil.copy("dataset/spall/accepted/" + spall_images[i], "dataset/spall/accepted/validate")
            else:
                shutil.copy("dataset/spall/accepted/" + spall_images[i], "dataset/spall/accepted/test")

        print("Copying images to train, test and validate folder completed")

    else:
        print("Images already in train, test and validate folder")
        
# load the images
crack_train_images = os.listdir("dataset/crack/accepted/train")
spall_train_images = os.listdir("dataset/spall/accepted/train")
print("Number of crack images: ", len(crack_train_images))
print("Number of spall images: ", len(spall_train_images))



Loading images ...
Number of crack images:  6541
Number of spall images:  8577
Number of crack images:  3925
Number of spall images:  5145


In [191]:
# function for extracting edges and contours from an image
def get_contours(image):
    # convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # apply gaussian blur
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    # apply canny edge detection
    canny = cv2.Canny(blur, 50, 150)
    # apply dilation
    kernel = np.ones((5, 5), np.uint8)
    dilated = cv2.dilate(canny, kernel, iterations=1)
    # apply erosion
    eroded = cv2.erode(dilated, kernel, iterations=1)
    # find contours
    contours, hierarchy = cv2.findContours(eroded, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    return contours

# function for drawing contours on an image
def draw_contours(image, contours):
    # draw contours on a copy of the image
    image_copy = np.copy(image)
    image_copy = cv2.drawContours(image_copy, contours, -1, (0, 255, 0), 3)
    return image_copy

# function for sorting contours from left to right
def x_cord_contour(contours):
    # return the x coordinate for the contour centroid
    if cv2.contourArea(contours) > 10:
        M = cv2.moments(contours)
        return (int(M['m10']/M['m00']))

# function for sorting contours from top to bottom
def label_contour_center(image, c):
    # place a red circle on the centers of contours
    M = cv2.moments(c)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    # draw the countour number on the image
    cv2.circle(image, (cx, cy), 10, (0, 0, 255), -1)
    return image

# function for creating a mask
def make_mask(image, contour):
    # fill contour
    mask = np.zeros_like(image)
    cv2.drawContours(mask, [contour], -1, 255, -1)
    # now crop
    (x, y) = np.where(mask == 255)
    (topx, topy) = (np.min(x), np.min(y))
    (bottomx, bottomy) = (np.max(x), np.max(y))
    cropped = image[topx:bottomx+1, topy:bottomy+1]
    return cropped


# function for sorting contours from top to bottom
def sort_contours(cnts, method="left-to-right"):
    reverse = False
    i = 0
    if method == "right-to-left" or method == "bottom-to-top":
        reverse = True
    if method == "top-to-bottom" or method == "bottom-to-top":
        i = 1
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
    key=lambda b:b[1][i], reverse=reverse))
    return (cnts, boundingBoxes)



# function for displaying images
def display(img, cmap='gray'):
    fig = plt.figure(figsize=(12,10))
    ax = fig.add_subplot(111)
    ax.imshow(img, cmap='gray')


In [200]:
# ET
import xml.etree.ElementTree as ET

# function to create annotation for images in VGG Image Annotator format
def create_annotation(image, contour, image_name):
    # get the image height and width
    height, width = image.shape[:2]
    # create the xml file
    annotation = ET.Element("annotation")
    ET.SubElement(annotation, "folder").text = "images"
    ET.SubElement(annotation, "filename").text = image_name
    ET.SubElement(annotation, "path").text = "images/" + image_name
    source = ET.SubElement(annotation, "source")
    ET.SubElement(source, "database").text = "Unknown"
    size = ET.SubElement(annotation, "size")
    ET.SubElement(size, "width").text = str(width)
    ET.SubElement(size, "height").text = str(height)
    ET.SubElement(size, "depth").text = "3"
    ET.SubElement(annotation, "segmented").text = "0"
    # create the object
    object = ET.SubElement(annotation, "object")
    ET.SubElement(object, "name").text = "crack"
    ET.SubElement(object, "pose").text = "Unspecified"
    ET.SubElement(object, "truncated").text = "0"
    ET.SubElement(object, "difficult").text = "0"
    bndbox = ET.SubElement(object, "bndbox")
    # get the bounding box coordinates
    x, y, w, h = cv2.boundingRect(contour)
    ET.SubElement(bndbox, "xmin").text = str(x)
    ET.SubElement(bndbox, "ymin").text = str(y)
    ET.SubElement(bndbox, "xmax").text = str(x + w)
    ET.SubElement(bndbox, "ymax").text = str(y + h)
    # create a new XML file with the results
    mydata = ET.tostring(annotation)
    # "via_project.json" for VGG Image Annotator
    folder_name = [crack, spall]
    for folder in folder_name:
        if folder == crack:
            mydata = ET.tostring(annotation)
            myfile = open("dataset/crack/accepted/train/annotations/" + image_name[:-4] + ".xml", "wb")
            myfile.write(mydata)
        elif folder == spall:
            mydata = ET.tostring(annotation)
            myfile = open("dataset/spall/accepted/train/annotations/" + image_name[:-4] + ".xml", "wb")
            myfile.write(mydata)
    
    return mydata    
    

In [202]:
# create function to annotate the images in train and create "via_project.json" for all images in train
# get the images in train from crack and spall
crack_images = os.listdir("dataset/crack/accepted/train")
spall_images = os.listdir("dataset/spall/accepted/train")

def annotate_images(dataset):
    # get the images in train from crack and spall
    for image_name in dataset:
        # read the image
        image = cv2.imread("dataset/crack/accepted/train/" + image_name)
        # convert to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # blur the image
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        # apply canny edge detection
        canny = cv2.Canny(blurred, 50, 150)
        # find contours
        contours, hierarchy = cv2.findContours(canny, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        # sort contours from left to right
        sorted_contours = sorted(contours, key=x_cord_contour, reverse=False)
        # create a copy of the image
        image_copy = np.copy(image)
        # draw contours on the image
        image_copy = draw_contours(image_copy, sorted_contours)
        # label the center of the contours
        image_copy = label_contour_center(image_copy, sorted_contours)
        # create a mask
        mask = make_mask(image, sorted_contours[0])
        # create annotation
        create_annotation(image, sorted_contours[0], image_name)
        # display the image
        display(image_copy)
        # display the mask
        display(mask)
    return dataset

In [203]:
# annotate the images in train
annotate_images(crack_images)
annotate_images(spall_images)


TypeError: '<' not supported between instances of 'NoneType' and 'NoneType'

In [ ]:
"""
# function to create annotations for each image and save it as a json file in the annotation folder
def create_annotation_crack(image_name, image_path, image, contours):
    # create a dictionary
    data = {}
    data['imagePath'] = image_path
    data['imageData'] = None
    data['imageHeight'] = image.shape[0]
    data['imageWidth'] = image.shape[1]
    data['shapes'] = []
    for i in range(len(contours)):
        shape = {}
        shape['label'] = 'crack'
        shape['points'] = []
        shape['group_id'] = None
        shape['shape_type'] = 'polygon'
        shape['flags'] = {}
        for j in range(len(contours[i])):
            shape['points'].append([int(contours[i][j][0][0]), int(contours[i][j][0][1])])
        data['shapes'].append(shape)
    # save the dictionary as a json file
    with open('dataset/crack/accepted/annotation/'+image_name+'.json', 'w') as outfile:
        json.dump(data, outfile)

# function to create annotations for each image and save it as a json file in the annotation folder
def create_annotation_spall(image_name, image_path, image, contours):
    # create a dictionary
    data = {}
    data['imagePath'] = image_path
    data['imageData'] = None
    data['imageHeight'] = image.shape[0]
    data['imageWidth'] = image.shape[1]
    data['shapes'] = []
    for i in range(len(contours)):
        shape = {}
        shape['label'] = 'spall'
        shape['points'] = []
        shape['group_id'] = None
        shape['shape_type'] = 'polygon'
        shape['flags'] = {}
        for j in range(len(contours[i])):
            shape['points'].append([int(contours[i][j][0][0]), int(contours[i][j][0][1])])
        data['shapes'].append(shape)
    # save the dictionary as a json file
    with open('dataset/spall/accepted/annotation/'+image_name+'.json', 'w') as outfile:
        json.dump(data, outfile)

"""

In [ ]:
# Root directory of the project
ROOT_DIR = os.path.abspath("../")
print(ROOT_DIR)

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library

# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

class CustomConfig(Config):
    """Configuration for training on the toy  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "custom"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + custom

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 131

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9

config = CustomConfig()
config.display()



c:\Users\DELL\Downloads

Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE               

In [ ]:
# create custom dataset
class CustomDataset(utils.Dataset):
        def load_custom(self, dataset_dir, subset):
            """Load a subset of the Custom dataset.
            dataset_dir: Root directory of the dataset.
            subset: Subset to load: train or val
            """
            # Add classes. We have only one class to add.
            self.add_class("custom", 1, "crack")
            self.add_class("custom", 2, "spall")
    
            # Train or validation dataset?
            assert subset in ["train", "val"]
            dataset_dir = os.path.join(dataset_dir, subset)
    
            # Load annotations
            # VGG Image Annotator (up to version 1.6) saves each image in the form:
            # { 'filename': '28503151_5b5b7ec140_b.jpg',
            #   'regions': {
            #       '0': {
            #           'region_attributes': {},
            #           'shape_attributes': {
            #               'all_points_x': [...],
            #               'all_points_y': [...],
            #               'name': 'polygon'}},
            #       ... more regions ...
            #   },
            #   'size': 100202
            # }
            # We mostly care about the x and y coordinates of each region
            
            # Load annotations from json file for each image in the 
            # crack/accepted/annotation and spall/accepted/annotation
            
            # function to load annotations for crack and spall images in accepted folder
            def load_annotations(dataset_dir, subset):
                # Add images
                for image_name in os.listdir(dataset_dir):
                    if image_name.endswith('.json'):
                        annotations = json.load(open(os.path.join(dataset_dir, image_name)))
                        annotations = list(annotations.values())
                        # some images don't have any annotations. Skip them.
                        annotations = [a for a in annotations if a['regions']]
                        for a in annotations:
                            polygons = [r['shape_attributes'] for r in a['regions'].values()]
                            name_dict = [r['region_attributes'] for r in a['regions'].values()]
                            name = [list(n.keys())[0] for n in name_dict]
                            objects = [s['name'] for s in polygons]
                            name_dict = dict(zip(objects, name))
                            num_ids = [1 if n == 'crack' else 2 for n in name]
                            num_ids = [int(n) for n in num_ids]
                            name_dict = {1: 'crack', 2: 'spall'}
                            print(name_dict)
                            # load_mask() needs the image size to convert polygons
                            # to masks. Unfortunately, VIA doesn't include it in
                            # JSON, so we must read the image. This is only managable
                            # since the dataset is tiny.
                            image_path = os.path.join(dataset_dir, a['filename'])
                            image = skimage.io.imread(image_path)
                            height, width = image.shape[:2]
    
                            self.add_image(
                                "custom",
                                image_id=a['filename'],  # use file name as a unique image id
                                path=image_path,
                                width=width, height=height,
                                polygons=polygons,
                                num_ids=num_ids,
                                name_dict=name_dict)


            # load annotations for crack and spall images in accepted folder
            load_annotations(dataset_dir, subset)

        def load_mask(self, image_id):
            """Generate instance masks for an image.
            Returns:
            masks: A bool array of shape [height, width, instance count] with
                one mask per instance.
            class_ids: a 1D array of
            """
            # If not a custom dataset image, delegate to parent class.
            image_info = self.image_info[image_id]
            if image_info["source"] != "custom":
                return super(self.__class__, self).load_mask(image_id)
            # Convert polygons to a bitmap mask of shape
            # [height, width, instance_count]
            info = self.image_info[image_id]
            num_ids = info['num_ids']
            name_dict = info['name_dict']
            mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                            dtype=np.uint8)

            for i, p in enumerate(info["polygons"]):
                # Get indexes of pixels inside the polygon and set them to 1
                rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
                mask[rr, cc, i] = 1

            num_ids = np.array(num_ids, dtype=np.int32)
            return mask, num_ids

        def image_reference(self, image_id):
            """Return the path of the image."""
            info = self.image_info[image_id]
            if info["source"] == "custom":
                return info["path"]
            else:
                super(self.__class__, self).image_reference(image_id)
                

In [ ]:
# function for the training dataset
def train(model):
    """Train the model."""
    # Training dataset.
    dataset_train = CustomDataset()
    dataset_train.load_custom(args.dataset, "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = CustomDataset()
    dataset_val.load_custom(args.dataset, "val")
    dataset_val.prepare()

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=40,
                layers='heads')


In [ ]:
# Training dataset
dataset_train = CustomDataset()
dataset_train.load_custom(args.dataset, "train")
dataset_train.prepare()


# Validation dataset
dataset_val = CustomDataset()
dataset_val.load_custom(dataset, "val")
dataset_val.prepare()

# Load and display random samples
image_ids = np.random.choice(dataset_train.image_ids, 4)
for image_id in image_ids:
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)

# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                            model_dir=args.logs)

# Which weights to start with?
init_with = "coco"  # imagenet, coco, or last

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_WEIGHTS_PATH, by_name=True,
                        exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])

elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last(), by_name=True)

# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=30,
            layers='heads')

# Fine tune all layers
# Passing layers="all" trains all layers. You can also
# pass a regular expression to select which layers to
# train by name pattern.
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE / 10,
            epochs=40,
            layers="all")

# Save weights
# Typically not needed because callbacks save after every epoch
# Uncomment to save manually
model_path = os.path.join(MODEL_DIR, "model/new_mask_rcnn_shapes.h5")
model.keras_model.save_weights(model_path)

NameError: name 'args' is not defined